In [1]:
import sys
sys.path.append('../..')

import numpy as np
import pandas as pd
import gru4rec
import evaluation

PATH_TO_TRAIN = 'processedrsc15_train_full.txt'
PATH_TO_TEST = 'processedrsc15_test.txt'
pd.options.display.max_rows = 10

In [ ]:
data = pd.read_csv(PATH_TO_TRAIN, sep='\t', dtype={'ItemId':np.int64})
valid = pd.read_csv(PATH_TO_TEST, sep='\t', dtype={'ItemId':np.int64})

In [ ]:
data.info()

In [ ]:
valid.info()

In [ ]:
gru = gru4rec.GRU4Rec(loss='top1-max',final_act='softmax',hidden_act='relu',
                      layers=[100],batch_size=50, dropout_p_hidden=0.5,
                      learning_rate=0.01,momentum=0.0,time_sort=False)

In [ ]:
%time gru.fit(data)

In [ ]:
res = evaluation.evaluate_sessions_batch(gru,valid,None)

In [ ]:
res

In [ ]:
gru = gru4rec.GRU4Rec(loss='bpr', final_act='tanh', hidden_act='tanh', 
                      layers=[100], batch_size=50, dropout_p_hidden=0.5, 
                      learning_rate=0.01, momentum=0.0, time_sort=False)

In [ ]:
%time gru.fit(data)

In [ ]:
res = evaluation.evaluate_sessions_batch(gru, valid, None)
print('Recall@20: {}'.format(res[0]))
print('MRR@20: {}'.format(res[1]))

In [ ]:
gru = gru4rec.GRU4Rec(loss='bpr-max-0.5', final_act='linear', hidden_act='tanh', 
                      layers=[100], batch_size=32, dropout_p_hidden=0.0, learning_rate=0.2, 
                      momentum=0.5, n_sample=2048, sample_alpha=0, time_sort=True)

In [ ]:
%time gru.fit(data)

In [ ]:
len(res)

In [ ]:
ls

In [2]:
dR = pd.read_csv('reddit_01_17_posts.csv',header=None)

In [3]:
dR.columns=['Time','UserName','Topic','SubReddit']

#### Drop all the nan's and drop the topic, remove the '[deleted]' usernames

In [4]:
dR.dropna(inplace=True)
dR.drop(['Topic'],axis=1,inplace=True)

In [153]:
dR = dR[dR.UserName != '[deleted]']

In [156]:
'[deleted]' in dR.UserName.unique()

False

#### Group by UserName and remove all users who made less than 3 contributions

In [157]:
m = dR.groupby('UserName').filter(lambda x: len(x)>=3)

In [160]:
m.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5779914 entries, 12 to 9218508
Data columns (total 3 columns):
Time         int64
UserName     object
SubReddit    object
dtypes: int64(1), object(2)
memory usage: 176.4+ MB


In [159]:
t = dR.groupby('Time').filter(lambda x: len(x)>=3)

KeyboardInterrupt: 

In [ ]:
m2 = m.set_index(['UserName','Time'])

In [ ]:
m2.sort_index(level=0)

In [ ]:
dR_sorted = m2.sort_index(level=0)

In [ ]:
dR_sorted.describe()

In [145]:
dR_sorted.xs(1483620068,level=1)

,SubReddit
UserName,
----------_-_-,MandelaEffect
[deleted],cats
edenikseres,greece


In [146]:
dR_sorted.xs('[deleted]')

,SubReddit
Time,
1483228800,AskReddit
1483228804,thenewcoldwar
1483228806,Fireteams
1483228809,GlobalOffensiveTrade
1483228809,NoFapChristians
...,...
1485907193,The_Donald
1485907196,Showerthoughts
1485907198,Flyers


In [151]:
print(dR_sorted.xs('[deleted]'))

                       SubReddit
Time                            
1483228800             AskReddit
1483228804         thenewcoldwar
1483228806             Fireteams
1483228809  GlobalOffensiveTrade
1483228809       NoFapChristians
...                          ...
1485907193            The_Donald
1485907196        Showerthoughts
1485907198                Flyers
1485907199             AskReddit
1485907199  GlobalOffensiveTrade

[1658432 rows x 1 columns]


In [152]:
dR_sorted = dR_sorted[dR_sorted.UserName != '[deleted]']

AttributeError: 'DataFrame' object has no attribute 'UserName'